### MovieLens 20M 데이터(gt)와 train_df를 비교하는 파일
##### gt가 아닐 수도 있긴 하지만..

In [14]:
import pandas as pd
import time
import os
from IPython.display import display_html
from tqdm import tqdm

In [15]:
#데이터 프레임 가로 출력 함수
def display_dfs(dfs, gap=50, justify='center'):
    html = ""
    for title, df in dfs.items():  
        df_html = df._repr_html_()
        cur_html = f'<div> <h3>{title}</h3> {df_html}</div>'
        html +=  cur_html
    html= f"""
    <div style="display:flex; gap:{gap}px; justify-content:{justify};">
        {html}
    </div>
    """
    display_html(html, raw=True)

In [16]:
# 전체 학습 데이터
data_path = 'data/train'
train_df = pd.read_csv(os.path.join(data_path, 'train_ratings.csv'))

In [17]:
# 무비렌즈 오픈 데이터셋 ml-20m, gt로 명명
gt_df = pd.read_csv('/opt/ml/input/ml-20m/ratings.csv')

In [18]:
# train 데이터셋에 있는 user와 item으로 필터링
isinfilter = gt_df['userId'].isin(train_df['user'].unique())
gt_df = gt_df[isinfilter]
isinfilter = gt_df['movieId'].isin(train_df['item'].unique())
gt_df = gt_df[isinfilter]

In [19]:
# timestamp datetime 객체로 변환
train_df['time'] = pd.to_datetime(train_df['time'].apply(lambda x:time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(x))))
gt_df['timestamp'] = pd.to_datetime(gt_df['timestamp'].apply(lambda x:time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(x))))

In [20]:
# train_df의 가장 과거 시간 확인
min(train_df['time'])

Timestamp('2005-04-11 11:56:25')

In [21]:
# train_df의 가장 과거 시간으로 필터링(안 하면 4개보다 차이나는 유저가 약 2700명 생김)
gt_df = gt_df[gt_df['timestamp']>='2005-04-11 11:56:25']

In [22]:
# 본 영화가 적어서 비교하기 쉬운 user 확인
train_df.groupby('user').count().sort_values('item')

,item,time
user,,
53188,16,16
68606,19,19
105578,22,22
128756,28,28
11211,32,32
...,...,...
91349,1795,1795
32514,1830,1830
24219,1842,1842


In [23]:
# user_id 넣어서 눈으로 비교하기(눈으로 봐도 4개 빼고 다 같음)
# 다른 4개 중 2개는 맨 뒤 2개
# 나머지 2개는 규칙성을 못 찾겠음(랜덤 같음)
user_id = 11
rating_cutoff = 3.5

temp = gt_df[gt_df['userId']==user_id].sort_values('timestamp')

print(f"train_df 길이: {len(train_df[train_df['user']==user_id])}")
print(f"gt_df 길이: {len(temp[temp['rating']>=rating_cutoff])}")

dfs = {'train_df':train_df[train_df['user']==user_id], 'gt_df':temp[temp['rating']>=rating_cutoff]}
display_dfs(dfs, justify='flex-start')

train_df 길이: 376
gt_df 길이: 380


,user,item,time
0,11,4643,2009-01-01 04:02:09
1,11,170,2009-01-01 04:02:14
2,11,531,2009-01-01 04:02:19
3,11,616,2009-01-01 04:02:22
4,11,2140,2009-01-01 04:02:43
...,...,...,...
371,11,48738,2011-01-12 01:35:06
372,11,6291,2011-01-12 01:35:13
373,11,46578,2011-01-12 01:35:19
374,11,7153,2011-01-12 01:35:32


In [80]:
# user별로 train_df와 gt_df간의 아이템 수 차이 구하기
gt_diff_cnt = {}
for user in tqdm(train_df['user'].unique()):
    temp = gt_df[gt_df['userId']==user]
    gt_diff_cnt[user] =  len(temp[temp['rating']>=rating_cutoff]) - len(train_df[train_df['user']==user])

100%|██████████| 31360/31360 [10:14<00:00, 51.00it/s]


In [81]:
# 데이터프레임화
diff_cnt_df = pd.DataFrame(list(gt_diff_cnt.items()), columns=['user', 'gt_diff_cnt'])

In [82]:
# 차이가 4개가 아닌 user 없음
diff_cnt_df[diff_cnt_df['gt_diff_cnt']!=4]

,user,gt_diff_cnt


In [13]:
train_df[(train_df['user']==11) &(train_df['item']==58025)]

,user,item,time


In [34]:
# # 유저별로 gt에만 있는 4개의 데이터를 담은 리스트를 csv로 저장
# diff_dic = {}
# for user in tqdm(train_df['user'].unique()):
#     train_temp = train_df[train_df['user']==user]
#     temp = gt_df[gt_df['userId']==user].sort_values('timestamp')
#     gt_temp = temp[temp['rating']>=rating_cutoff]

#     diff_dic[user] = list(set(gt_temp['movieId'].unique()) - set(train_temp['item'].unique()))

# diff_df = pd.DataFrame(list(diff_dic.items()), columns=['user', 'diff'])
# diff_df.to_csv('diff_movie4.csv', index=False)

100%|██████████| 31360/31360 [10:31<00:00, 49.65it/s]
